<a href="https://colab.research.google.com/github/wendychen215/aop113b/blob/main/EX04_06_ET_Today_%E6%96%B0%E8%81%9E%E6%A8%99%E9%A1%8C%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

爬取過去一年 ET Today 的新聞標題做為資料集。

使用 Jieba 進行斷詞。

使用單純貝氏演算法進行新聞標題分類。

## 資料收集

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-ettoday_news.csv"
df = pd.read_csv(url)
df.head()

,類別,標題
0,政治,青年座談不被AI取代的關鍵力 葉丙成：只有這兩種人才能真正把握時間紅利
1,政治,蕭美琴直播初體驗曝「咪琴嚴選」 邀網友挺花蓮
2,政治,疑似國軍「在美受訓」照曝光！ 陸軍不評論
3,政治,「蔡英文、柯文哲有談NCC」 府：沒談人選、政黨比例更沒私下喬
4,政治,總統府否認蔡英文向柯喬NCC委員 黃國昌嗆：醜聞被揭露還敢說謊


## 資料前處理

### 資料清理

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29216 entries, 0 to 29215
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   類別      29216 non-null  object
 1   標題      29216 non-null  object
dtypes: object(2)
memory usage: 456.6+ KB


### 探索性分析

### 資料分割

In [ ]:
!wget -O dict.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
!wget -O stopwords.txt https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt

--2025-06-06 01:40:04--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-dict.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4154480 (4.0M) [application/octet-stream]
Saving to: ‘dict.txt’

dict.txt            100%[===================>]   3.96M  --.-KB/s    in 0.07s   

2025-06-06 01:40:04 (60.8 MB/s) - ‘dict.txt’ saved [4154480/4154480]

--2025-06-06 01:40:04--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-stopwords.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8129 (7.9

In [ ]:
import jieba
jieba.set_dictionary('dict.txt')
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = set(line.strip() for line in f if line.strip())
    return stopwords
def jieba_cut_with_stopwords(text, stopwords):
    words = jieba.cut(text)
    return [word for word in words if word not in stopwords and word.strip() != '']
stopwords = load_stopwords("stopwords.txt")

In [ ]:
df["標題"] = df["標題"].apply(lambda x: " ".join(jieba_cut_with_stopwords(x, stopwords)))

Building prefix dict from /content/dict.txt ...
DEBUG:jieba:Building prefix dict from /content/dict.txt ...
Loading model from cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.ueb620ec8402181953a0299d7957c0d6e.cache
Loading model cost 0.998 seconds.
DEBUG:jieba:Loading model cost 0.998 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
df

,類別,標題
0,政治,青年 座談 AI 取代 關鍵 力 葉丙成 這兩種 人才 真正 把握 時間 紅利
1,政治,蕭美琴 直播 體驗 曝 咪琴 嚴選 邀 網友 挺 花蓮
2,政治,疑似 國軍 美 受訓 曝光 陸軍 評論
3,政治,蔡英文 柯文哲 有談 NCC 府 沒談 人選 政黨 比例 私下 喬
4,政治,總統府 否認 蔡英文 柯喬 NCC 委員 黃國昌 嗆 醜聞 揭露 還敢 說謊
...,...,...
29211,生活,瞞 未婚夫 下海 拍片 休息 月 後 I 級 AV 女優 宣布 引退 原因 曝
29212,生活,努力 考上 清大 家人 句 嫁 有錢 工程師 心冷
29213,生活,北榮 藝廊 閃 一週 陳威明 鎮館 寶 古柏 園 象徵 醫者 精神
29214,生活,收到 一整 串 粽子 媽 氣炸 大票 愣 每年 吃到 吐 沒聽過


In [ ]:
X, y = df["標題"], df["類別"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 類別轉換

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### 特徵縮放

## 模型訓練

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=1.0) # alpha 就是設定如果該詞不存在時，因貝氐運算是採用乘法，所以用 1
mnb.fit(X_train, y_train)

MultinomialNB()

In [ ]:
mnb.score(X_train, y_train)

0.910909002004792

## 模型評估

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = mnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7759269823160296


## 模型調整

## 模型部署

### 儲存模型

### 推論預測

In [ ]:
text = "AKIRA放浪兄弟震撼彈！　2人氣男神宣布停止活動…辭退巨蛋巡演"
s = vectorizer.transform([" ".join(jieba_cut_with_stopwords(text, stopwords))])
c = mnb.predict(s)[0]
category = label_encoder.classes_[c]
print(category)

影劇
